In [1]:
pip install PyMuPDF pdfplumber pandas xlsxwriter


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.0/58.0 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 17.5 MB/s eta 0:00:00


In [24]:
import os
import zipfile
import fitz  # PyMuPDF
import pdfplumber
import pandas as pd
import re
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def extract_text_pymupdf(pdf_path):
    """Extract text from a PDF file using PyMuPDF."""
    try:
        doc = fitz.open(pdf_path)
        text = ""
        for page_num in range(len(doc)):
            page = doc.load_page(page_num)
            text += page.get_text()
        return text
    except Exception as e:
        logging.error(f"Error extracting text from {pdf_path}: {e}")
        return ""

def extract_invoice_details(text):
    """Extract invoice details from text using regex."""
    details = {
        "Sold By": "",
        "Billing Address": "",
        "PAN No": "",
        "Shipping Address": "",
        "GST Registration No": "",
        "Order Number": "",
        "Order Date": "",
        "Place of supply": "",
        "Place of delivery": "",
        "Invoice Number": "",
        "Invoice Date": ""
    }

    regex_patterns = {
        "Sold By": r"Sold By\s*:\s*([^\n]+)",
        "PAN No": r"PAN No\s*:\s*(\S+)",
        "GST Registration No": r"GST Registration No\s*:\s*(\S+)",
        "Order Number": r"Order Number\s*:\s*(\S+)",
        "Order Date": r"Order Date\s*:\s*(\S+)",
        "Place of supply": r"Place of supply\s*:\s*(\S+)",
        "Place of delivery": r"Place of delivery\s*:\s*(\S+)",
        "Invoice Number": r"Invoice Number\s*:\s*(\S+)",
        "Invoice Date": r"Invoice Date\s*:\s*(\S+)"
    }

    for key, pattern in regex_patterns.items():
        match = re.search(pattern, text)
        if match:
            details[key] = match.group(1).strip()

    billing_address_match = re.search(r"Billing Address\s*:\s*(.*?)Shipping Address\s*:", text, re.DOTALL)
    if billing_address_match:
        details["Billing Address"] = billing_address_match.group(1).strip()

    shipping_address_match = re.search(r"Shipping Address\s*:\s*(.*?)$", text, re.DOTALL)
    if shipping_address_match:
        details["Shipping Address"] = shipping_address_match.group(1).strip()

    return details

def extract_tables_pdfplumber(pdf_path):
    """Extract tables from a PDF file using pdfplumber."""
    table1 = []
    table2 = []

    try:
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                page_tables = page.extract_tables()
                for table in page_tables:
                    if table:
                        if len(table1) == 0:
                            header = table[0]
                            for row in table[1:]:
                                row_dict = dict(zip(header, row))
                                table1.append(row_dict)
                        else:
                            for row in table:
                                table2.append(row)
    except Exception as e:
        logging.error(f"Error extracting tables from {pdf_path}: {e}")

    return table1, table2

def process_zip(zip_path, save_dir, excel_file):
    """Process each PDF in the ZIP file and save extracted data to a single Excel file with multiple sheets."""
    try:
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            pdf_files = [file for file in zip_ref.namelist() if file.endswith('.pdf')]

            all_invoice_details = []
            all_table1 = []
            all_table2 = []

            for pdf_file in pdf_files:
                # Ensure the directory exists
                file_dir = os.path.join(save_dir, os.path.dirname(pdf_file))
                os.makedirs(file_dir, exist_ok=True)

                with zip_ref.open(pdf_file) as file:
                    pdf_path = os.path.join(save_dir, pdf_file)
                    with open(pdf_path, 'wb') as f:
                        f.write(file.read())

                    logging.info(f"Processing {pdf_file}...")
                    text = extract_text_pymupdf(pdf_path)
                    invoice_details = extract_invoice_details(text)
                    table1, table2 = extract_tables_pdfplumber(pdf_path)

                    all_invoice_details.append(invoice_details)
                    all_table1.extend(table1)
                    all_table2.extend(table2)

            df_invoice_details = pd.DataFrame(all_invoice_details)
            df_invoice_details.columns = [
                "Sold By", "Billing Address", "PAN No", "Shipping Address",
                "GST Registration No", "Order Number", "Order Date",
                "Place of supply", "Place of delivery", "Invoice Number",
                "Invoice Date"
            ]

            df_table1 = pd.DataFrame(all_table1)
            df_table2 = pd.DataFrame(all_table2, columns=["Payment Transaction ID", 'Date & Time', 'Invoice Value', 'Mode of Payment'])

            with pd.ExcelWriter(excel_file, engine='xlsxwriter') as writer:
                df_invoice_details.to_excel(writer, sheet_name='Order Details', index=False)
                df_table1.to_excel(writer, sheet_name='Invoices Table1', index=False)
                df_table2.to_excel(writer, sheet_name='Invoices Table2', index=False)

            logging.info(f"All extracted data has been saved to {excel_file}.")

    except Exception as e:
        logging.error(f"Error processing ZIP file {zip_path}: {e}")


zip_path = '/content/AMAZON1.zip'
save_dir = './AMZN/'
os.makedirs(save_dir, exist_ok=True)
excel_file = os.path.join(save_dir, 'amzn_combined.xlsx')
process_zip(zip_path, save_dir, excel_file)
